In [1]:
from langfuse.callback import CallbackHandler
from dotenv import load_dotenv
import os
load_dotenv()

public_key = os.getenv("ENV_PUBLIC_KEY", None)
secret_key = os.getenv("ENV_SECRET_KEY", None)
env_host = os.getenv("ENV_HOST", None)

print(public_key, secret_key, env_host)

pk-lf-97ba6052-895e-4696-9ddc-07f7ab2e0d2b sk-lf-ca78cef9-37c4-4a19-8398-fe4c07b21de5 http://localhost:3000


In [2]:
from langfuse import Langfuse
 
langfuse = Langfuse(public_key, secret_key, env_host, debug=True)

In [3]:
from datetime import datetime
from langfuse.model import InitialGeneration, Usage
 
generationStartTime = datetime.now()
 
# call to an LLM API
 
langfuse.generation(InitialGeneration(
    name="summary-generation",
    startTime=generationStartTime,
    endTime=datetime.now(),
    model="gpt-3.5-turbo",
    modelParameters={"maxTokens": "1000", "temperature": "0.9"},
    prompt=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals..."}],
    completion="The Q3 OKRs contain goals for multiple teams...",
    usage=Usage(promptTokens=50, completionTokens = 49),
    metadata={"interface": "whatsapp"}
))

In [4]:
from langfuse.model import CreateTrace, CreateSpan, CreateGeneration, CreateEvent
 
trace = langfuse.trace(CreateTrace(name = "llm-feature"))
retrieval = trace.span(CreateSpan(name = "retrieval"))
retrieval.generation(CreateGeneration(name = "query-creation"))
retrieval.span(CreateSpan(name = "vector-db-search"))
retrieval.event(CreateEvent(name = "db-summary"))
trace.generation(CreateGeneration(name = "user-output"))

In [41]:
from langfuse.model import CreateTrace
 
trace = langfuse.trace(CreateTrace(
    name = "docs-retrieval",
    userId = "user__935d7d1d-8625-4ef4-8651-544613e7bd22",
    metadata = {
        "env": "production",
        "email": "user@langfuse.com",
    }
))

In [26]:
from datetime import datetime
from langfuse.model import CreateSpan, UpdateSpan
 
retrievalStartTime = datetime.now()
 
# retrieveDocs = retrieveDoc()
# ...
 
span = trace.span(CreateSpan(
        name="embedding-search",
        startTime=retrievalStartTime,
        endTime=datetime.now(),
        metadata={"database": "pinecone"},
        input = {'query': 'This document entails the OKR goals for ACME'},
    )
)

In [27]:
span = span.update(UpdateSpan(
        output = {"response": "[{'name': 'OKR Engineering', 'content': 'The engineering department defined the following OKR goals...'},{'name': 'OKR Marketing', 'content': 'The marketing department defined the following OKR goals...'}]"}
    )
)

In [9]:
from langfuse.model import CreateGeneration, Usage, UpdateGeneration
 
from datetime import datetime
 
generationStartTime = datetime.now()
 
# chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
# ...
 
generation = trace.generation(CreateGeneration(
    name="summary-generation",
    startTime=generationStartTime,
    endTime=datetime.now(),
    model="gpt-3.5-turbo",
    modelParameters={"maxTokens": "1000", "temperature": "0.9"},
    prompt=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals..."}],
    metadata={"interface": "whatsapp"}
))

In [10]:
generation.update(UpdateGeneration(
    completion="The Q3 OKRs contain goals for multiple teams...",
    usage=Usage(promptTokens=50, completionTokens = 49),
))

In [11]:
from langfuse.model import CreateEvent
from datetime import datetime
 
event = span.event(CreateEvent(
        name="chat-docs-retrieval",
        startTime=datetime.now(),
        metadata={"key": "value"},
        input = {"key": "value"},
        output = {"key": "value"}
    )
)

In [12]:
from langfuse.model import CreateScore
 
trace.score(CreateScore(
    name="user-explicit-feedback",
    value=1,
    comment="I like how personalized the response is"
))

In [15]:
!pip install fastapi

In [16]:
from contextlib import asynccontextmanager
from fastapi import FastAPI, Query, BackgroundTasks
from langfuse.model import InitialGeneration
 
 
@asynccontextmanager
async def lifespan(app: FastAPI):
    # Operation on startup
 
    yield  # wait until shutdown
 
    # Flush all events to be sent to Langfuse on shutdown and terminate all Threads gracefully. This operation is blocking.
    langfuse.flush()
 
 
app = FastAPI(lifespan=lifespan)

In [17]:
langfuse = Langfuse(public_key, secret_key, env_host, debug=True)
 
@app.get("/generate/",tags=["APIs"])
async def campaign(prompt: str = Query(..., max_length=20)):
  # call to a LLM
  generation = langfuse.generation(
      InitialGeneration(name="llm-feature", metadata="test", prompt=prompt)
  )
  return True